In [2]:
import torch
from torch_geometric.datasets import DBLP

import torch_geometric.transforms as T
import torch.nn.functional as F
import graph_polluters
import graph_learning
import apriori
import autoencoder



path = './data/dblp'
# We initialize conference node features with a single one-vector as feature:
dataset = DBLP(path, transform=T.Constant(node_types='conference'))
data = dataset[0]
print(data)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

SyntaxError: invalid syntax (graph_polluters.py, line 36)

In [5]:
torch.sum(data['author'].x, 1)

tensor([ 7., 22.,  7.,  ...,  0.,  0.,  4.])

# Baseline

In [8]:
graph_learning.set_seed()
dataset_copy = dataset.copy()
data_copy = dataset_copy[0]
data_copy.to(device)
model, optimizer = graph_learning.init_parameters(data_copy, device)
train_accs, val_accs, test_accs = [],[],[]
for epoch in range(1, 100):
    loss = graph_learning.train(data=data_copy, model=model, optimizer=optimizer)
    train_acc, val_acc, test_acc = graph_learning.test(data = data_copy, model=model)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    test_accs.append(test_acc)
best_epoch = max(enumerate(val_accs),key=lambda x: x[1])[0]
train_acc, val_acc, test_acc = train_accs[best_epoch], val_accs[best_epoch], test_accs[best_epoch]
print(f'Baseline, Train: {train_acc:.4f}, '
        f'Val: {val_acc:.4f}, Test: {test_acc:.4f}')

Baseline, Train: 1.0000, Val: 0.8025, Test: 0.8090


# 50% features removed

In [7]:
graph_learning.set_seed()

#copy the dataset
dataset_copy = dataset.copy()
data_copy = dataset_copy[0]
data_copy.to(device)
data_copy.cpu()

# remove features
data_copy = graph_polluters.remove_features(data_copy, 0.5)

model, optimizer = graph_learning.init_parameters(data_copy, device)
train_accs, val_accs, test_accs = [],[],[]
for epoch in range(1, 100):
    loss = graph_learning.train(data=data_copy, model=model, optimizer=optimizer)
    train_acc, val_acc, test_acc = graph_learning.test(data = data_copy, model=model)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    test_accs.append(test_acc)
best_epoch = max(enumerate(val_accs),key=lambda x: x[1])[0]
train_acc, val_acc, test_acc = train_accs[best_epoch], val_accs[best_epoch], test_accs[best_epoch]
print(f'Baseline, Train: {train_acc:.4f}, '
        f'Val: {val_acc:.4f}, Test: {test_acc:.4f}')

Baseline, Train: 1.0000, Val: 0.7150, Test: 0.7283


# 50% features removed + apriori added

In [9]:
graph_learning.set_seed()

#copy the dataset
dataset_copy = dataset.copy()
data_copy = dataset_copy[0]
#data_copy.to(device)

data_copy = graph_polluters.remove_features(data_copy, 0.5)

item_sets = apriori.find_frequent_sets(data_copy, 'author', min_support = 0.01)
apriori.conflate_item_sets(data_copy, 'author', item_sets)
# remove features
# add new features

model, optimizer = graph_learning.init_parameters(data_copy,device)
train_accs, val_accs, test_accs = [],[],[]
for epoch in range(1, 100):
    loss = graph_learning.train(data=data_copy, model=model, optimizer=optimizer)
    train_acc, val_acc, test_acc = graph_learning.test(data = data_copy, model=model)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    test_accs.append(test_acc)
best_epoch = max(enumerate(val_accs),key=lambda x: x[1])[0]
train_acc, val_acc, test_acc = train_accs[best_epoch], val_accs[best_epoch], test_accs[best_epoch]
print(f'Baseline, Train: {train_acc:.4f}, '
        f'Val: {val_acc:.4f}, Test: {test_acc:.4f}')

Baseline, Train: 1.0000, Val: 0.7900, Test: 0.7937


# 50% features removed + autoencoded

In [5]:
graph_learning.set_seed()

#copy the dataset
dataset_copy = dataset.copy()
data_copy = dataset_copy[0]
data_copy.to(device)

# remove features
data_copy = graph_polluters.remove_features(data_copy, 0.5, ['author'])

# add new features


model, optimizer = graph_learning.init_parameters(data_copy, device)
train_accs, val_accs, test_accs = [],[],[]
for epoch in range(1, 100):
    loss = graph_learning.train(data=data_copy, model=model, optimizer=optimizer)
    train_acc, val_acc, test_acc = graph_learning.test(data = data_copy, model=model)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    test_accs.append(test_acc)
best_epoch = max(enumerate(val_accs),key=lambda x: x[1])[0]
train_acc, val_acc, test_acc = train_accs[best_epoch], val_accs[best_epoch], test_accs[best_epoch]
print(f'Baseline, Train: {train_acc:.4f}, '
        f'Val: {val_acc:.4f}, Test: {test_acc:.4f}')

Baseline, Train: 1.0000, Val: 0.7800, Test: 0.7915


In [3]:
graph_learning.set_seed()

#copy the dataset
dataset_copy = dataset.copy()
data_copy = dataset_copy[0]
#data_copy.to(device)

item_sets = apriori.find_frequent_sets(data_copy, 'author', min_support = 0.02)
print(item_sets)
apriori.conflate_item_sets(data_copy, 'author', item_sets)

KeyboardInterrupt: 